# Project Briefing Generator Cookbook

A comprehensive guide for building an AI-powered project briefing system using CAMEL-AI and Mem0 for intelligent knowledge management and context-aware briefing generation.

## Overview

This cookbook demonstrates how to create a system that can:
- Store and organize project knowledge using vector-based memory
- Generate role-specific project briefings for new team members
- Answer contextual questions about project decisions and history
- Manage project memories with semantic search capabilities

## Prerequisites

### Dependencies

In [ ]:
%%bash
# Core dependencies
pip install mem0ai camel-ai python-dotenv


### Environment Setup
Create a `.env` file with your API keys:
```env
OPENAI_API_KEY=your_openai_api_key_here
```

## Core Architecture

### 1. System Initialization

In [ ]:
import os
import time
from typing import List, Dict
from mem0 import Memory
from camel.agents import ChatAgent
from camel.messages import BaseMessage
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType
from dotenv import load_dotenv

class ProjectBriefingSystem:
    def __init__(self):
        # Load environment variables
        load_dotenv()
        
        # Initialize Mem0 with fallback options
        self.memory = self._initialize_memory()
        
        # Initialize CAMEL agents
        self.knowledge_agent = self._create_agent(
            "Knowledge Extractor", 
            "You extract and organize project knowledge from documents and conversations."
        )
        
        self.briefing_agent = self._create_agent(
            "Briefing Generator",
            "You create comprehensive project briefings for new team members."
        )
        
        self.qa_agent = self._create_agent(
            "Q&A Assistant",
            "You answer specific questions about project decisions and context."
        )

### 2. Memory Initialization with Fallback

In [ ]:
def _initialize_memory(self):
    """Initialize Mem0 with multiple fallback strategies"""
    try:
        # Method 1: Simple initialization
        return Memory()
    except Exception as e1:
        print(f"Default initialization failed: {e1}")
        
        try:
            # Method 2: With basic configuration
            config = {
                "vector_store": {
                    "provider": "chroma",
                    "config": {
                        "collection_name": "project_briefings"
                    }
                }
            }
            return Memory(config)
        except Exception as e2:
            print(f"Configured initialization failed: {e2}")
            
            # Method 3: Fallback to in-memory storage
            print("Using fallback in-memory storage...")
            self.memory_store = {}
            return None

def _create_agent(self, role: str, description: str) -> ChatAgent:
    """Create a CAMEL agent with specified role and description"""
    model = ModelFactory.create(
        model_platform=ModelPlatformType.OPENAI,
        model_type=ModelType.GPT_4O_MINI
    )
    
    return ChatAgent(
        system_message=BaseMessage.make_assistant_message(
            role_name=role,
            content=description
        ),
        model=model
    )

## Core Features

### 3. Storing Project Knowledge

In [ ]:
def store_project_info(self, content: str, project_id: str, content_type: str):
    """Store project information with metadata"""
    metadata = {
        "project_id": project_id,
        "content_type": content_type,
        "timestamp": time.time(),
        "date": time.strftime("%Y-%m-%d", time.localtime())
    }
    
    if self.memory is not None:
        # Use Mem0 vector storage
        try:
            result = self.memory.add(content, user_id=project_id, metadata=metadata)
            print(f"✅ Information stored in Mem0: {result}")
            return result
        except Exception as e:
            # Try alternative format
            try:
                result = self.memory.add(
                    messages=[{"role": "user", "content": content}], 
                    user_id=project_id, 
                    metadata=metadata
                )
                print(f"✅ Information stored (messages format): {result}")
                return result
            except Exception as e2:
                # Simple add without metadata
                result = self.memory.add(content, user_id=project_id)
                print(f"✅ Information stored (simple): {result}")
                return result
    else:
        # Use fallback storage
        key = f"{project_id}_{content_type}_{int(time.time())}"
        self.memory_store[key] = {
            "content": content,
            "project_id": project_id,
            "content_type": content_type,
            "timestamp": time.time()
        }
        print("✅ Information stored in fallback memory")
        return {"message": "Stored in fallback memory"}

### 4. Retrieving Contextual Information

In [ ]:
def retrieve_project_context(self, query: str, project_id: str, limit: int = 10) -> List[Dict]:
    """Retrieve relevant project context using semantic search"""
    if self.memory is not None:
        try:
            # Use Mem0 semantic search
            results = self.memory.search(query=query, user_id=project_id, limit=limit)
            if results:
                print(f"📊 Found {len(results)} relevant memories")
                return results
            else:
                print("No relevant memories found")
                return []
        except Exception as e:
            print(f"Search failed: {e}")
            # Fallback to getting all memories and filtering
            try:
                all_memories = self.memory.get_all(user_id=project_id)
                if all_memories:
                    # Simple keyword filtering
                    query_words = query.lower().split()
                    filtered = []
                    for item in all_memories:
                        content = str(item.get('memory', '') or item.get('text', '')).lower()
                        if any(word in content for word in query_words):
                            filtered.append(item)
                    print(f"📊 Found {len(filtered)} memories (keyword search)")
                    return filtered[:limit]
                return []
            except Exception as e2:
                print(f"Fallback search failed: {e2}")
                return []
    else:
        # Use fallback storage with keyword search
        results = []
        query_words = query.lower().split()
        
        for key, data in self.memory_store.items():
            if data["project_id"] == project_id:
                content_lower = data["content"].lower()
                score = sum(content_lower.count(word) for word in query_words if word in content_lower)
                
                if score > 0:
                    results.append({
                        "memory": data["content"],
                        "score": score,
                        "metadata": {
                            "content_type": data["content_type"],
                            "timestamp": data.get("timestamp", 0)
                        }
                    })
        
        results.sort(key=lambda x: x["score"], reverse=True)
        print(f"📊 Found {len(results)} relevant memories from fallback")
        return results[:limit]

### 5. Generating Project Briefings

In [ ]:
def generate_briefing(self, project_id: str, new_member_role: str) -> str:
    """Generate comprehensive project briefing for new team members"""
    
    # Define search queries for comprehensive context
    context_queries = [
        f"project overview {new_member_role}",
        "key decisions rationale",
        "timeline milestones",
        "team structure contacts",
        "technical architecture",
        "current status challenges"
    ]
    
    # Gather context from multiple searches
    all_context = []
    for query in context_queries:
        context = self.retrieve_project_context(query, project_id, limit=5)
        all_context.extend(context)
    
    # Remove duplicates
    seen_content = set()
    unique_context = []
    for item in all_context:
        content = item.get('memory', '') or item.get('text', '') or str(item)
        if content not in seen_content and content.strip():
            seen_content.add(content)
            unique_context.append(item)
    
    # Build context text
    context_text = ""
    if unique_context:
        for item in unique_context:
            memory_text = item.get('memory', '') or item.get('text', '') or str(item)
            metadata = item.get('metadata', {})
            content_type = metadata.get('content_type', 'Unknown')
            date = metadata.get('date', 'Unknown date')
            context_text += f"[{content_type} - {date}]: {memory_text}\n\n"
    else:
        context_text = "No previous project information found. This appears to be a new project."
    
    # Generate briefing prompt
    prompt = f"""
    Generate a comprehensive project briefing for a new {new_member_role} joining the team.
    
    Project Context (from project memory):
    {context_text}
    
    Create a well-structured briefing that includes:
    1. **Project Overview** - What is this project about?
    2. **Key Decisions & Rationale** - Important decisions made and why
    3. **Technical Architecture** - Tech stack and architectural decisions
    4. **Team Structure** - Who's working on what
    5. **Current Status** - Where we are now
    6. **Challenges & Blockers** - Current issues to be aware of
    7. **Next Steps** - What the new {new_member_role} should focus on
    8. **Important Contacts** - Key people to connect with
    9. **Resources & Documentation** - Where to find more information
    
    Make it specific to a {new_member_role} role and actionable.
    """
    
    try:
        response = self.briefing_agent.step(BaseMessage.make_user_message(
            role_name="User",
            content=prompt
        ))
        return response.msg.content
    except Exception as e:
        return f"Error generating briefing: {str(e)}"

### 6. Question Answering System

In [ ]:
def answer_question(self, question: str, project_id: str) -> str:
    """Answer specific questions about the project using contextual memory"""
    
    # Search for relevant context
    context = self.retrieve_project_context(question, project_id, limit=8)
    
    # Build context from search results
    context_text = ""
    if context:
        for item in context:
            memory_text = item.get('memory', '') or item.get('text', '') or str(item)
            metadata = item.get('metadata', {})
            content_type = metadata.get('content_type', 'Unknown')
            date = metadata.get('date', 'Unknown date')
            context_text += f"[{content_type} - {date}]: {memory_text}\n\n"
    else:
        context_text = "No relevant project information found for this question."
    
    prompt = f"""
    Answer this question based on the project context from memory:
    
    **Question:** {question}
    
    **Project Context:**
    {context_text}
    
    Provide a detailed, accurate answer based on the available information. 
    If the context doesn't contain enough information to answer completely, 
    mention what information is missing and suggest where it might be found.
    
    Include specific details and reasoning from the project memory when available.
    """
    
    try:
        response = self.qa_agent.step(BaseMessage.make_user_message(
            role_name="User",
            content=prompt
        ))
        return response.msg.content
    except Exception as e:
        return f"Error answering question: {str(e)}"

## Usage Examples

### Basic Usage

In [ ]:
# Initialize the system
briefing_system = ProjectBriefingSystem()

# Store project knowledge
project_id = "project_alpha"

# Add various types of project information
briefing_system.store_project_info(
    content="Decided to migrate from MongoDB to PostgreSQL for better ACID compliance. Migration scheduled for Q2 2024.",
    project_id=project_id,
    content_type="Decision Log"
)

briefing_system.store_project_info(
    content="Team structure: 3 backend developers, 2 frontend developers, 1 DevOps engineer, 1 Product Manager",
    project_id=project_id,
    content_type="Team Update"
)

briefing_system.store_project_info(
    content="Architecture uses microservices with Docker containers, deployed on AWS EKS",
    project_id=project_id,
    content_type="Technical Doc"
)

# Generate a briefing for a new developer
briefing = briefing_system.generate_briefing(project_id, "Backend Developer")
print("Generated Briefing:")
print(briefing)

# Ask specific questions
answer = briefing_system.answer_question(
    "Why did we switch from MongoDB to PostgreSQL?", 
    project_id
)
print("\nQ&A Answer:")
print(answer)

### Batch Processing Example

In [ ]:
def batch_store_project_data(briefing_system, project_id: str, data_entries: List[Dict]):
    """Store multiple project entries at once"""
    results = []
    for entry in data_entries:
        try:
            result = briefing_system.store_project_info(
                content=entry['content'],
                project_id=project_id,
                content_type=entry['type']
            )
            results.append({"success": True, "result": result})
        except Exception as e:
            results.append({"success": False, "error": str(e)})
    return results

# Example batch data
project_data = [
    {
        "content": "Sprint 1 completed. Delivered user authentication and basic dashboard.",
        "type": "Sprint Update"
    },
    {
        "content": "API rate limiting implemented using Redis. Max 1000 requests per hour per user.",
        "type": "Technical Decision"
    },
    {
        "content": "New team member Sarah joins as Senior Frontend Developer on March 1st.",
        "type": "Team Update"
    }
]

# Store batch data
results = batch_store_project_data(briefing_system, "project_beta", project_data)

### Advanced Memory Management

In [ ]:
def get_all_project_memories(self, project_id: str) -> List[Dict]:
    """Retrieve all memories for a specific project"""
    try:
        if self.memory is not None:
            memories = self.memory.get_all(user_id=project_id)
            return memories if memories else []
        else:
            # Use fallback storage
            project_memories = []
            for key, data in self.memory_store.items():
                if data["project_id"] == project_id:
                    project_memories.append({
                        "memory": data["content"],
                        "metadata": {
                            "content_type": data["content_type"],
                            "timestamp": data.get("timestamp", 0)
                        }
                    })
            return project_memories
    except Exception as e:
        print(f"Error getting memories: {str(e)}")
        return []

def delete_memory(self, memory_id: str, project_id: str) -> bool:
    """Delete a specific memory"""
    try:
        if self.memory is not None:
            result = self.memory.delete(memory_id=memory_id, user_id=project_id)
            return True
        else:
            # Delete from fallback storage
            keys_to_delete = [
                key for key, data in self.memory_store.items()
                if data["project_id"] == project_id and str(data.get("timestamp", "")) == memory_id
            ]
            for key in keys_to_delete:
                del self.memory_store[key]
            return len(keys_to_delete) > 0
    except Exception as e:
        print(f"Error deleting memory: {str(e)}")
        return False

## Best Practices

### Content Structuring

In [ ]:
# Good: Structured content with context
content = """
Decision: Switched from REST to GraphQL for API
Date: 2024-01-15
Reason: Better data fetching efficiency, reduced over-fetching
Impact: Frontend team needs to update client libraries
Stakeholders: Backend team, Frontend team, Product team
Timeline: Implementation by end of Q1 2024
"""

# Better: Include reasoning and next steps
content = """
Technical Decision - API Architecture Change

What: Migrated from REST API to GraphQL
When: January 15, 2024
Why: 
- Reduced over-fetching by 40%
- Better developer experience for frontend team
- Improved mobile app performance

Impact:
- All frontend clients need updates
- New GraphQL playground for testing
- Updated documentation required

Next Steps:
- Frontend team training scheduled for Jan 22
- Client library migration by Feb 1
- Legacy REST endpoints deprecated by March 1
"""

### Effective Questioning

In [ ]:
# Examples of effective questions for the Q&A system
effective_questions = [
    "Why did we choose technology X over Y?",
    "What were the main challenges in the last sprint?",
    "Who should I contact about the database migration?",
    "What is our current deployment process?",
    "What are the known performance bottlenecks?",
    "When is the next major release planned?",
    "What security considerations were discussed?",
    "How do we handle error monitoring?",
    "What are the testing strategies in place?",
    "Who are the key stakeholders for feature X?"
]

### Memory Organization

In [ ]:
# Organize content by types for better retrieval
content_types = {
    "Decision Log": "Important project decisions and rationale",
    "Meeting Notes": "Meeting summaries and action items",
    "Technical Doc": "Architecture, design, and implementation details",
    "Team Update": "Team changes, roles, and responsibilities", 
    "Sprint Update": "Sprint goals, achievements, and blockers",
    "Architecture Decision": "Technical architecture choices",
    "Process Change": "Changes to development or operational processes",
    "Incident Report": "System issues and resolutions"
}

## Error Handling and Debugging

In [ ]:
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def debug_memory_operations(self, operation: str, **kwargs):
    """Debug helper for memory operations"""
    logger.info(f"Memory operation: {operation}")
    logger.info(f"Parameters: {kwargs}")
    
    if self.memory is None:
        logger.warning("Using fallback memory storage")
    else:
        logger.info("Using Mem0 vector storage")

def validate_inputs(self, content: str, project_id: str, content_type: str):
    """Validate inputs before storing"""
    if not content or not content.strip():
        raise ValueError("Content cannot be empty")
    
    if not project_id or not project_id.strip():
        raise ValueError("Project ID cannot be empty")
    
    if len(content) < 10:
        logger.warning("Content is very short, consider adding more detail")
    
    return True

## Troubleshooting

### Common Issues and Solutions

1. **Memory initialization fails**
   ```python
   # The system includes automatic fallback to in-memory storage
   # Check your environment variables and API keys
   ```

2. **Poor search results**
   ```python
   # Use more specific search terms
   # Include context and keywords relevant to your query
   # Consider breaking complex questions into simpler ones
   ```

3. **Agent responses are generic**
   ```python
   # Store more detailed and structured project information
   # Include specific dates, people, and technical details
   # Use consistent terminology across your project knowledge
   ```

## Extending the System

### Custom Agents

In [ ]:
def create_custom_agent(self, role: str, system_prompt: str) -> ChatAgent:
    """Create a custom agent for specific tasks"""
    model = ModelFactory.create(
        model_platform=ModelPlatformType.OPENAI,
        model_type=ModelType.GPT_4O_MINI
    )
    
    return ChatAgent(
        system_message=BaseMessage.make_assistant_message(
            role_name=role,
            content=system_prompt
        ),
        model=model
    )

# Example: Risk Assessment Agent
risk_agent = create_custom_agent(
    "Risk Assessor",
    "You analyze project information to identify potential risks and suggest mitigation strategies."
)

### Integration with External Systems

In [ ]:
def integrate_with_slack(self, webhook_url: str, briefing: str):
    """Send briefing to Slack channel"""
    import requests
    
    payload = {
        "text": f"📋 New Project Briefing Generated:\n\n{briefing}"
    }
    
    response = requests.post(webhook_url, json=payload)
    return response.status_code == 200

def export_to_markdown(self, briefing: str, filename: str):
    """Export briefing to markdown file"""
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(briefing)
    print(f"Briefing exported to {filename}")

This cookbook provides a complete foundation for building an intelligent project briefing system using CAMEL-AI and Mem0, with robust error handling, fallback mechanisms, and extensibility options.